# Import Libraries

In [1]:
import pandas as pd
import requests
import yfinance as yf
import numpy as np
from pathlib import Path

# Get Data
### We get the quarterly financial and stock price data from yahoo finance.
### We will only keep the companies in the S&P500 with data since 2020-01-01

In [2]:
# URL of the Wikipedia page containing the S&P 500 companies
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies";

# Define HTTP headers to simulate a real browser
# This helps avoid being blocked by the website (HTTP 403)
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/117.0.0.0 Safari/537.36"
    )
}

# Send an HTTP GET request with custom headers
response = requests.get(url, headers=headers)

# Raise an exception if the request failed (status code not 200)
response.raise_for_status()

# Parse all HTML tables from the downloaded page
# pandas.read_html can parse tables from a string of HTML
tables = pd.read_html(response.text)

# The first table on the page contains the S&P 500 companies
sp500_table = tables[0]

# Extract the 'Symbol' column to get the list of tickers
tickers = sp500_table['Symbol'].tolist()

# Print the first 10 tickers as a sanity check
print(tickers[:10])
print (f'Number of Tickers: {len(tickers)}')


/var/folders/m3/_s6_yj8n00sdydg5rx4z5fjh0000gn/T/ipykernel_16458/2154613338.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']
Number of Tickers: 503


In [3]:
#We download closing price of all tickers since 2020-01-01
prices = yf.download(tickers, start='2020-01-01')['Close']

print(f'prices shape: {prices.shape}')

#We keep only tickers that have data since 2020-01-01
prices = prices.dropna(axis=1)

#We store the tickers in the DataFrame 'data'
tickers = prices.columns.tolist()

print(f'prices shape: {prices.shape}')

print(f'Number of tickers with closing price data since 2020-01-01 to current date: {len(tickers)}')

#We stack the values in 'data' to reorganize the DataFrame so we have columns 'Date', 'Ticker', 'close_stock_price'
prices = pd.DataFrame(prices.stack()).reset_index().sort_values(['Ticker', 'Date'])
prices.rename(columns={0: 'close_stock_price'}, inplace=True)

print(f'prices shape: {prices.shape}')

/var/folders/m3/_s6_yj8n00sdydg5rx4z5fjh0000gn/T/ipykernel_16458/2502048630.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers, start='2020-01-01')['Close']
[*********************100%***********************]  503 of 503 completed

3 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-12-22)')
['GM']: Timeout('Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


prices shape: (1502, 503)
prices shape: (1502, 472)
Number of tickers with closing price data since 2020-01-01 to current date: 472
prices shape: (708944, 3)


In [4]:
# Create a financials dataframe with all quarterly financial statements of tickers with closing price data since 2020-01-01

financials_list = []   # Create an empty list. Each element of the list will be a financials DF for each ticker
failed_tickers = []    # Track errors

for tkr in tickers:
    try:
        ticker = yf.Ticker(tkr)
        income_q = ticker.quarterly_income_stmt         # Income statement, quarterly
        balance_q = ticker.quarterly_balance_sheet      # Balance sheet, quarterly
        cashflow_q = ticker.quarterly_cashflow          # Cash flow, quarterly
        
        # Validate that they are not empty 
        if income_q.empty or balance_q.empty or cashflow_q.empty:
            print(f"⚠️ {tkr}: Empty financial statements, skipping")
            failed_tickers.append(tkr)
            continue
        
        # Combine all the financial data transposed (dates as row index and financials as columns)
        financials_tkr = pd.concat([income_q.T, balance_q.T, cashflow_q.T], axis=1)


        financials_tkr.reset_index(inplace=True)      # We reset the index
        financials_tkr.columns.values[0] = 'Date'     # We rename the 1st column (old index) 'Date'
        financials_tkr.insert(0, 'Ticker', tkr)       # Insert a column 'Ticker' at the beginning of the DataFrame to identify the company

        # Add the financials DF to the list
        financials_list.append(financials_tkr)
        
    except Exception as e:
        print(f"❌ {tkr}: Failed with error: {e}")
        failed_tickers.append(tkr)
        continue  # Continue loop with next ticker

financials = pd.concat(financials_list, axis=0, ignore_index=True)      # Concat the financials for all the tickers.

# Final report
print(f"\n✓ Success: {len(financials_list)} tickers")
print(f"✗ Failed: {len(failed_tickers)} tickers")
if failed_tickers:
    print(f"Failed list: {failed_tickers}")


✓ Success: 472 tickers
✗ Failed: 0 tickers


In [ ]:
# Merge the DataFrames 'financials' and 'prices'

# Get sure that Dates are in the correct format.
financials['Date'] = pd.to_datetime(financials['Date'])
prices['Date'] = pd.to_datetime(prices['Date'])

# Create a new column price_date where we add 5 days to the column 'Date'
# We do that in order to fix the look-ahead bias (stock prices react after some time of publishing financial results)
financials['price_date'] = financials['Date'] + pd.Timedelta(days=5)

# Merge using adjusted date. In case of not having a price for a date, we will use the next days price with a tolerance of 10 days.
financials = pd.merge_asof(
    financials.sort_values('price_date'),
    prices.rename(columns={'Date': 'price_date'}).sort_values('price_date'),
    on='price_date',
    by='Ticker',
    direction='forward',
    tolerance=pd.Timedelta(days=10)
)

# We rename the index with ticker and date info so each observation is easier to track in the future
financials = financials.set_index(
    financials['Ticker'] + "_" + financials['Date'].dt.strftime('%Y-%m-%d')
)

financials.sort_values(['Ticker', 'Date'], inplace=True)

/var/folders/m3/_s6_yj8n00sdydg5rx4z5fjh0000gn/T/ipykernel_16458/4269354273.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  financials['price_date'] = financials['Date'] + pd.Timedelta(days=5)


# We save the data as a pickle in the folder 'data'

In [6]:
#We save the 'financials' DataFrame as a pickle with the name raw_financials.pkl in the 'data' folder
#We save the data folder path or create it if it doesn't exist
project_path = Path('..')          
data_folder = project_path / 'data' 
data_folder.mkdir(exist_ok=True) 

#We save the DataFrame in pikle format
file_path = data_folder / 'raw_financials.pkl'
financials.to_pickle(file_path)

print(f"DataFrame saved successfully in {file_path}")


DataFrame saved successfully in ../data/raw_financials.pkl
